# Proceso de Machine Learning

## Selección del mejor Dataset

In [12]:
import pandas as pd
import numpy as np
from pickle import dump

from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import RandomizedSearchCV

#Silenciar los warnings 
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

X_train = pd.read_excel("../data/processed/X_train.xlsx")
X_test = pd.read_excel("../data/processed/X_test.xlsx")

y_train = pd.read_excel("../data/processed/y_train.xlsx")
y_test = pd.read_excel("../data/processed/y_test.xlsx")

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train["review"]).toarray()
X_test = vec_model.transform(X_test["review"]).toarray()


y_train = y_train.squeeze()
y_test = y_test.squeeze()

diccionario_modelos={
    "Guaussiano": GaussianNB(),
    "Multinomial": MultinomialNB(),
    "Bernoulli": BernoulliNB()
}

for nombre_modelo, tipo_modelo in diccionario_modelos.items():
    model = tipo_modelo
    model.fit(X_train, y_train)

    y_pred = model.predict(X_train)

    print(f"La métrica del modelo {nombre_modelo} es: \n{accuracy_score(y_train, y_pred)}")

La métrica del modelo Guaussiano es: 
0.9859550561797753
La métrica del modelo Multinomial es: 
0.9592696629213483
La métrica del modelo Bernoulli es: 
0.922752808988764


## Optimización de hiperparámetros

In [13]:
parametros_modelos = {
    "modelo_Guaussiano": {
        "modelo": GaussianNB(),
        "params": {
                       "var_smoothing": np.logspace(0, -9, num=100)
                  }
    },

    "modelo_Multinomial": {
        "modelo": MultinomialNB(),
        "params": {
                       "alpha": [0.001, 0.01, 0.1, 0.5, 1.0],
                       "fit_prior": [True, False]
                  }
    },

    "modelo_Bernoulli": {
        "modelo": BernoulliNB(),
        "params": {
                       "alpha": [0.001, 0.01, 0.1, 0.5, 1.0],
                       "fit_prior": [True, False],
                       "binarize": [0.0, 0.1, 0.5, 1.0]
                  }
    }                        
}

### Random search

In [14]:

best_scored_model = 0
best_model_gr = ""
name_model = ""

for name, info in parametros_modelos.items():
    grid = RandomizedSearchCV(info["modelo"], info["params"], scoring="accuracy", n_iter = 300, n_jobs = -1)
    grid.fit(X_train, y_train)

    if best_scored_model < grid.best_score_:
        best_scored_model = grid.best_score_
        best_model_rnd = grid.best_estimator_
        name_model = name

print("El mejor modelo es: ", name_model)
print("El mejor estimador es: ", best_model_rnd)
print("La mejor puntuación es: ", best_scored_model)

El mejor modelo es:  modelo_Multinomial
El mejor estimador es:  MultinomialNB(alpha=0.1)
La mejor puntuación es:  0.8034078597458878


## Entrenar el mejor modelo

In [15]:
model_f = MultinomialNB(alpha=0.1)
model_f.fit(X_train, y_train)

y_pred = model_f.predict(X_train)
metric_train = accuracy_score(y_train, y_pred)


y_pred = model_f.predict(X_test)
metric_test = accuracy_score(y_test, y_pred)


print(f"La mejor métrica de nuestros datasets x_train son: {metric_train} y los x_test son: {metric_test}")

#Guardado de modelo

dump(model, open("../models/modelo_entrenado_Multinomial_analizador_sentimientos.sav", "wb"))


La mejor métrica de nuestros datasets x_train son: 0.9845505617977528 y los x_test son: 0.7821229050279329


 Podríamos utilizar varios modelos como Random Forest, Boosting Algorithm o Linear Regression pero cada uno tiene sus especificaciones y en que ámbito podrían ser más acordes al resultado que se 
está buscando ya que LR es mejor cuando hay correlacion entre las palabras y BA o RF cuando combinas texto con variables estructuradas.

### Random Forest

In [17]:
params = {
    "n_estimators": [200, 400],
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [3, 5],
    "min_samples_split": [2, 4], 
    "max_features": [None, "sqrt", "log2"],
}

grid_rf = RandomizedSearchCV(RandomForestClassifier(random_state=10), params, scoring="accuracy", n_iter = 30)
grid_rf.fit(X_train, y_train)

best_model = grid_rf.best_estimator_
print("El mejor estimador es: ", best_model)
print("La mejor puntuación es: ", grid.best_score_)

model_rf = best_model
model_rf.fit(X_train, y_train)

y_pred = model_rf.predict(X_train)
metric_train = accuracy_score(y_train, y_pred)


y_pred = model_rf.predict(X_test)
metric_test = accuracy_score(y_test, y_pred)


print(f"La mejor métrica de nuestros datasets x_train son: {metric_train} y los x_test son: {metric_test}")

El mejor estimador es:  RandomForestClassifier(criterion='entropy', max_depth=5, max_features=None,
                       n_estimators=400, random_state=10)
La mejor puntuación es:  0.8020388062641584
La mejor métrica de nuestros datasets x_train son: 0.8174157303370787 y los x_test son: 0.7150837988826816


Se puede apreciar que, para este caso, el Naive Bayes da mejores resultados pero los dados por el Random Forest tienen muy buenas métricas.